In [1]:
name = 'tests'

In [2]:
import os
import re
from pyspark.sql import SparkSession
from typing import List, Dict, Callable
import socket

spark = None

EXECUTOR_ENV = 'hdfs:///share/products/cvm5/lib/python/anaconda_2.4.4_ds.tar.gz'  # 2.4.4 
SPARK_ARCHIVE = 'hdfs:///share/lib/spark/sparkjars-2.4.4.zip'                     # 2.4.4
#EXECUTOR_ENV = 'hdfs:///share/lib/python/env/anaconda-2019.07.tar.gz'
#SPARK_ARCHIVE = 'hdfs:///share/lib/spark/sparkjars-2.3.1.zip'

os.environ["ARROW_LIBHDFS_DIR"] = "/usr/hdp/2.6.5.0-292/usr/lib"
os.environ['HADOOP_HOME'] = '/usr/hdp/current/hadoop-client/'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64/'
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf/'
os.environ['SPARK_HOME'] = '/opt/conda/lib/python3.7/site-packages/pyspark'
os.environ['PYSPARK_PYTHON'] = 'anaconda_2.4.4_ds.tar.gz/bin/python3'             # 2.4.4
#os.environ['PYSPARK_PYTHON'] = 'anaconda-2019.07.tar.gz/bin/python3'






def restart_spark(task_name: str, num_executors: int, executor_memory='4G', executor_cores=2,
                  driver_memory='2G', queue='cvm5-rnd', additional_params: Dict[str, str] = None):
    global spark

    if spark:
        sc = spark.sparkContext
        if sc and sc._jsc:
            if not sc._jsc.sc().isStopped():
                print('Using cached spark')
                return sc, spark

    need_ports_for_app = 3
    user_tcp_ports = _get_user_tcp_ports()
    free_ports = _get_free_ports(user_tcp_ports)
    assert len(free_ports) >= need_ports_for_app, \
        f"Not enough free ports ({len(free_ports)}), need {need_ports_for_app}, stop other apps"
    app_ports = free_ports[:need_ports_for_app]

    host_ip = os.getenv('HOST_IP')
    
    spark_session = (
        SparkSession
        .builder
        .appName(task_name)
        .master('yarn')
        .config('spark.driver.memory', driver_memory)
        .config('spark.driver.maxResultSize', driver_memory)
        .config('spark.executor.cores', executor_cores)
        .config('spark.executor.memory', executor_memory)
        .config('spark.executor.memoryOverhead', '1G')
        .config('spark.dynamicAllocation.enabled', 'true')
        .config('spark.dynamicAllocation.maxExecutors', num_executors)
        .config('spark.sql.broadcastTimeout', '36000')
        .config('spark.dynamicAllocation.cachedExecutorIdleTimeout', '1200s')
        .config('spark.ui.port', app_ports[0])
        .config('spark.blockManager.port', app_ports[1])
        .config('spark.driver.port', app_ports[2])
        .config('spark.driver.host', host_ip)
        .config('spark.driver.bindAddress', '0.0.0.0')
        .config('spark.driver.extraLibraryPath', '/usr/hdp/2.6.5.0-292/hadoop/lib/native')
        .config('spark.driver.extraJavaOptions', '-Dhdp.version=current')
        .config('spark.debug.maxToStringFields', '50')
        .config('spark.yarn.queue', queue)
        .config('spark.yarn.dist.archives', EXECUTOR_ENV)
        .config('spark.yarn.archive', SPARK_ARCHIVE)
        .config('spark.yarn.am.extraJavaOptions', '-Dhdp.version=current')
        .config('spark.rpc.message.maxSize', '1024')
        .config('spark.sql.warehouse.dir', '/apps/hive/warehouse')
        .config('spark.sql.execution.pandas.respectSessionTimeZone', 'false')
        .config('spark.sql.orc.filterPushdown', 'true')
        .config('spark.sql.hive.convertMetastoreOrc', 'true')
        .config('spark.shuffle.service.enabled', 'true')
        .config('spark.hadoop.yarn.timeline-service.enabled', 'false')
        .config('spark.hadoop.yarn.client.failover-proxy-provider',
                'org.apache.hadoop.yarn.client.ConfiguredRMFailoverProxyProvider')
        .config('spark.serializer', 'org.apache.spark.serializer.KryoSerializer')
        .config('spark.kryoserializer.buffer.max', '128m')
        .config('spark.executor.extraLibraryPath', '/usr/hdp/2.6.5.0-292/hadoop/lib/native')
    )

    if additional_params:
        for key, value in additional_params.items():
            spark_session = spark_session.config(key, value)

    spark = (
        spark_session
        .enableHiveSupport()
        .getOrCreate()
    )
    sc = spark.sparkContext

    return sc, spark


def _get_user_tcp_ports() -> List[str]:
    regexp = re.compile(r'-2e')
    envuser= os.getenv('HOSTNAME')
    if regexp.search(envuser):       
      _, user_name, user_surname = envuser.upper().split('-')
      user_full_name = '_'.join([user_name, user_surname])
    else:
      _, user_name  = envuser.upper().split('-') 
      user_full_name = user_name
    user_tcp_ports = [v for k, v in os.environ.items() if user_full_name in k and k.endswith('TCP_PORT')]
    return user_tcp_ports


def _get_free_ports(ports: List[str]):
    free_ports = []
    for port in ports:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(('0.0.0.0', int(port))) != 0:
                free_ports.append(port)
    return free_ports


In [3]:
sc, spark = restart_spark(
                    name, 
                    21, 
                    executor_memory='5G', 
                    executor_cores=3, 
                    driver_memory='7G', 
                    additional_params={"spark.sql.shuffle.partitions": "300"})

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
sc.setLogLevel('ERROR')

In [5]:
import pandas as pd
import sys
from pyspark.sql import functions as F
import datetime
from datetime import timedelta

sys.path.append('/home/jovyan/glow-byte-filters-pyspark')
from logic_filters import * 
from segmentation import *

In [6]:
LOYALTY_CARDS = "hive_ssa_tc5.loyalty_card"
LOYALTY_CARDHOLDERS = "hive_ssa_tc5.loyalty_cardholder"
ACCOUNTS = "hive_ssa_tc5.account"
CVM5_GUESTS = "hive_cvm_acrm.cvm5_guests"

DIM_STORE = "hive_ssa_main.dim_store"
CHECKS_HEADERS = "hive_ssa_main.fct_rtl_txn"
CHECKS_ITEMS = "hive_ssa_main.fct_rtl_txn_item"
PRODUCTS = "hive_ssa_tc5.cvm_product"

### Выбираем гостей нужного юзкейса

In [7]:
usecase = ['churn', 'frequency', 'cross', 'upgrade', 'ump']
dt = datetime.date(2021, 12, 22)

In [8]:
customers_usecase = (spark
                     .table(CVM5_GUESTS)
                     .filter(F.to_date('calculation_dt') == dt)
                     .filter(F.col('usecase').isin(usecase))
                     .select('account_no', 'customer_rk')
                            )

In [9]:
customers_usecase.count()

49418569

In [10]:
# seg_sms.write.parquet('temp2398_1', mode='overwrite')
seg_sms = spark.read.parquet('temp2398_1')

In [11]:
seg_sms.count() #24 145 153

25145153

In [12]:
favorite_store = spark.sql('''
with last_purch_tab as (
select l.loyalty_cardholder_acrm_id as customer_rk, max(checks.rtl_txn_start_dttm) as last_purch
from hive_ssa_main.fct_rtl_txn checks
join (select ch.loyalty_cardholder_acrm_id, c.loyalty_card_id from hive_ssa_tc5.loyalty_card c left join hive_ssa_tc5.loyalty_cardholder ch
on c.loyalty_account_id = ch.loyalty_account_id) l ON checks.loyalty_card_no = l.loyalty_card_id
where checks.financial_unit_format_dk ='D' and checks.rtl_txn_cancel_flg = 0
group by l.loyalty_cardholder_acrm_id
),
count_purch_tab as (
select l.loyalty_cardholder_acrm_id as customer_rk, checks.store_id, count(checks.rtl_txn_id) as count_purch,
max(checks.rtl_txn_start_dttm) as last_purch_dt
from hive_ssa_main.fct_rtl_txn checks
join (select ch.loyalty_cardholder_acrm_id, c.loyalty_card_id from hive_ssa_tc5.loyalty_card c
    left join hive_ssa_tc5.loyalty_cardholder ch
    on c.loyalty_account_id = ch.loyalty_account_id) l ON checks.loyalty_card_no = l.loyalty_card_id
join last_purch_tab lpt on lpt.customer_rk = l.loyalty_cardholder_acrm_id
where checks.financial_unit_format_dk ='D' and checks.rtl_txn_cancel_flg = 0 and
datediff(lpt.last_purch, checks.rtl_txn_dt) between 0 and 30
group by l.loyalty_cardholder_acrm_id, checks.store_id
),
max_purch_tab as (select customer_rk, max(count_purch) as max_count_purch, max(last_purch_dt) as last_purch
from count_purch_tab
group by customer_rk
)
select mpt.customer_rk, cpt.store_id
from max_purch_tab mpt
join count_purch_tab cpt on mpt.customer_rk = cpt.customer_rk and mpt.max_count_purch = cpt.count_purch
and mpt.last_purch = cpt.last_purch_dt
''')

In [13]:
active_users = seg_sms.join(favorite_store, 'customer_rk', 'inner')

In [14]:
stores_spb = (
    spark.table(DIM_STORE)
         .filter(F.col('federal_subject_dk').isin(['78']))
         .withColumn('federal_subject', F.lit('spb'))
         .select('store_id', 'federal_subject')
)
stores_msk = (
    spark.table(DIM_STORE)
         .filter(F.col('federal_subject_dk').isin(['77']))
         .withColumn('federal_subject', F.lit('msk'))
         .select('store_id', 'federal_subject')
)
stores_ug = (
    spark.table(DIM_STORE)
         .filter(F.col('macroregion_dk').isin(['MRDUG']))
         .withColumn('federal_subject', F.lit('ug'))
         .select('store_id', 'federal_subject')
)

In [15]:
stores = stores_spb.union(stores_msk).union(stores_ug)

In [16]:
count_people = (
      active_users.join(stores, 'store_id', 'inner')
                  .groupby('federal_subject').agg(F.countDistinct('customer_rk').alias('qty_customers'))
)

In [17]:
count_people_pd = count_people.toPandas()

In [20]:
count_people_pd

,federal_subject,qty_customers
0,ug,2847095
1,msk,1810295
2,spb,1014377


In [21]:
count_people_pd.to_csv('count_people.csv', index=False)